In [150]:
import tensorflow as tf

# Enable eager execution explicitly (in case it's turned off)
tf.config.run_functions_eagerly(True)

print(f"Eager execution enabled: {tf.executing_eagerly()}")


Eager execution enabled: True


In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, ReLU, BatchNormalization
from tensorflow.keras.models import Model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
def load_image(file_path):
    # Load and normalize image to [-1, 1]
    img = tf.io.read_file(file_path)  # Direct file path
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [256, 256])  # Resize to fixed size
    img = (tf.cast(img, tf.float32) / 127.5) - 1  # Normalize to [-1, 1]
    return img

def load_data(path, batch_size=16):
    files = glob(os.path.join(path, "*.png"))
    print(f"Files found in {path}: {files}")  # Debugging: Check file paths
    dataset = tf.data.Dataset.from_tensor_slices(files)
    dataset = dataset.map(load_image).batch(batch_size).shuffle(buffer_size=1000)
    return dataset


# Paths
data_path = r"E:\ideas\music2\music2_spectrograms"
train_piano_path = os.path.join(data_path, "Train_submission", "Sound_Piano")
train_guitar_path = os.path.join(data_path, "Train_submission", "Sound_Guitar")

# Load datasets
piano_dataset = load_data(train_piano_path)
guitar_dataset = load_data(train_guitar_path)

Files found in E:\ideas\music2\music2_spectrograms\Train_submission\Sound_Piano: ['E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\029500_morning-rain-piano-65875.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\035568_upright-piano-71570.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\another-sadmosphere-108461.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\arpeggio-01-36024.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\beautiful-random-minor-arp-119378.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\boogie-woogie-logo-116102.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\c-aeol-12984.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\cold-wednesday-73830.png', 'E:\\ideas\\music2\\music2_spectrograms\\Train_submission\\Sound_Piano\\converted_to_wav_file.pn

In [4]:
# Define CycleGAN Architecture
#Generator
def build_generator():
    inputs = Input(shape=[256, 256, 3])

    # Encoder
    x = Conv2D(64, (4, 4), strides=2, padding="same")(inputs)
    x = LeakyReLU()(x)
    x = Conv2D(128, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    # Bottleneck
    x = Conv2D(256, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    # Decoder
    x = Conv2DTranspose(128, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2DTranspose(64, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    outputs = Conv2DTranspose(3, (4, 4), strides=2, padding="same", activation="tanh")(x)
    return Model(inputs, outputs)

In [5]:
#Discriminator
def build_discriminator():
    inputs = Input(shape=[256, 256, 3])
    x = Conv2D(64, (4, 4), strides=2, padding="same")(inputs)
    x = LeakyReLU()(x)
    x = Conv2D(128, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(256, (4, 4), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(1, (4, 4), strides=1, padding="same")(x)
    return Model(inputs, x)

In [6]:
#Define Loss Functions
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real, fake):
    real_loss = loss_obj(tf.ones_like(real), real)
    fake_loss = loss_obj(tf.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5

def generator_loss(fake):
    return loss_obj(tf.ones_like(fake), fake)

def cycle_consistency_loss(real, cycled):
    return tf.reduce_mean(tf.abs(real - cycled))

def identity_loss(real, same):
    return tf.reduce_mean(tf.abs(real - same))

In [7]:
#Build and Compile the Model
# Initialize models
generator_piano_to_guitar = build_generator()
generator_guitar_to_piano = build_generator()
discriminator_piano = build_discriminator()
discriminator_guitar = build_discriminator()

# Optimizers
gen_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

previous training code

In [157]:
# #Training Loop
# @tf.function
# def train_step(real_piano, real_guitar):
#     with tf.GradientTape(persistent=True) as tape:
#         # Generate fake images
#         fake_guitar = generator_piano_to_guitar(real_piano, training=True)
#         fake_piano = generator_guitar_to_piano(real_guitar, training=True)

#         # Cycle consistency
#         cycled_piano = generator_guitar_to_piano(fake_guitar, training=True)
#         cycled_guitar = generator_piano_to_guitar(fake_piano, training=True)

#         # Identity mapping
#         same_piano = generator_guitar_to_piano(real_piano, training=True)
#         same_guitar = generator_piano_to_guitar(real_guitar, training=True)

#         # Discriminator outputs
#         disc_real_piano = discriminator_piano(real_piano, training=True)
#         disc_fake_piano = discriminator_piano(fake_piano, training=True)

#         disc_real_guitar = discriminator_guitar(real_guitar, training=True)
#         disc_fake_guitar = discriminator_guitar(fake_guitar, training=True)

#         # Losses
#         gen_piano_to_guitar_loss = generator_loss(disc_fake_guitar) + cycle_consistency_loss(real_piano, cycled_piano) + identity_loss(real_piano, same_piano)
#         gen_guitar_to_piano_loss = generator_loss(disc_fake_piano) + cycle_consistency_loss(real_guitar, cycled_guitar) + identity_loss(real_guitar, same_guitar)

#         disc_piano_loss = discriminator_loss(disc_real_piano, disc_fake_piano)
#         disc_guitar_loss = discriminator_loss(disc_real_guitar, disc_fake_guitar)

#     # Apply gradients
#     gen_gradients = tape.gradient([gen_piano_to_guitar_loss, gen_guitar_to_piano_loss], 
#                                    [generator_piano_to_guitar.trainable_variables, generator_guitar_to_piano.trainable_variables])
#     disc_gradients = tape.gradient([disc_piano_loss, disc_guitar_loss], 
#                                     [discriminator_piano.trainable_variables, discriminator_guitar.trainable_variables])

#     gen_optimizer.apply_gradients(zip(gen_gradients[0], generator_piano_to_guitar.trainable_variables))
#     gen_optimizer.apply_gradients(zip(gen_gradients[1], generator_guitar_to_piano.trainable_variables))
#     disc_optimizer.apply_gradients(zip(disc_gradients[0], discriminator_piano.trainable_variables))
#     disc_optimizer.apply_gradients(zip(disc_gradients[1], discriminator_guitar.trainable_variables))

In [8]:
# Separate optimizers for each generator and discriminator
gen_optimizer_piano_to_guitar = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
gen_optimizer_guitar_to_piano = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_optimizer_piano = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_optimizer_guitar = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

@tf.function
def train_step(real_piano, real_guitar):
    with tf.GradientTape(persistent=True) as tape:
        # Generate fake images
        fake_guitar = generator_piano_to_guitar(real_piano, training=True)
        fake_piano = generator_guitar_to_piano(real_guitar, training=True)

        # Cycle consistency
        cycled_piano = generator_guitar_to_piano(fake_guitar, training=True)
        cycled_guitar = generator_piano_to_guitar(fake_piano, training=True)

        # Identity mapping
        same_piano = generator_guitar_to_piano(real_piano, training=True)
        same_guitar = generator_piano_to_guitar(real_guitar, training=True)

        # Discriminator outputs
        disc_real_piano = discriminator_piano(real_piano, training=True)
        disc_fake_piano = discriminator_piano(fake_piano, training=True)

        disc_real_guitar = discriminator_guitar(real_guitar, training=True)
        disc_fake_guitar = discriminator_guitar(fake_guitar, training=True)

        # Losses
        gen_piano_to_guitar_loss = generator_loss(disc_fake_guitar) + cycle_consistency_loss(real_piano, cycled_piano) + identity_loss(real_piano, same_piano)
        gen_guitar_to_piano_loss = generator_loss(disc_fake_piano) + cycle_consistency_loss(real_guitar, cycled_guitar) + identity_loss(real_guitar, same_guitar)

        disc_piano_loss = discriminator_loss(disc_real_piano, disc_fake_piano)
        disc_guitar_loss = discriminator_loss(disc_real_guitar, disc_fake_guitar)

    # Compute and apply gradients for generators
    gen_piano_to_guitar_gradients = tape.gradient(gen_piano_to_guitar_loss, generator_piano_to_guitar.trainable_variables)
    gen_guitar_to_piano_gradients = tape.gradient(gen_guitar_to_piano_loss, generator_guitar_to_piano.trainable_variables)
    gen_optimizer_piano_to_guitar.apply_gradients(zip(gen_piano_to_guitar_gradients, generator_piano_to_guitar.trainable_variables))
    gen_optimizer_guitar_to_piano.apply_gradients(zip(gen_guitar_to_piano_gradients, generator_guitar_to_piano.trainable_variables))

    # Compute and apply gradients for discriminators
    disc_piano_gradients = tape.gradient(disc_piano_loss, discriminator_piano.trainable_variables)
    disc_guitar_gradients = tape.gradient(disc_guitar_loss, discriminator_guitar.trainable_variables)
    disc_optimizer_piano.apply_gradients(zip(disc_piano_gradients, discriminator_piano.trainable_variables))
    disc_optimizer_guitar.apply_gradients(zip(disc_guitar_gradients, discriminator_guitar.trainable_variables))

    del tape  # Free up memory


In [9]:
#Train the Model
EPOCHS = 50

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    progress_bar = tqdm(total=len(piano_dataset), desc="Batch Progress", unit="batch")
    
    for real_piano, real_guitar in zip(piano_dataset, guitar_dataset):
        train_step(real_piano, real_guitar)
        progress_bar.update(1)
    
    progress_bar.close()
    print(f"Epoch {epoch + 1}/{EPOCHS} completed.")




Epoch 1/50


Batch Progress: 100%|██████████| 33/33 [02:50<00:00,  5.18s/batch]


Epoch 1/50 completed.

Epoch 2/50


Batch Progress: 100%|██████████| 33/33 [03:29<00:00,  6.35s/batch]


Epoch 2/50 completed.

Epoch 3/50


...

Epoch 25/50 completed.

Epoch 26/50


Batch Progress: 100%|██████████| 33/33 [04:25<00:00,  8.06s/batch]


Epoch 26/50 completed.

Epoch 27/50


Batch Progress:   0%|          | 0/33 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved models
generator_piano_to_guitar = load_model("generator_piano_to_guitar.h5")
generator_guitar_to_piano = load_model("generator_guitar_to_piano.h5")
